In [1]:
# workaround for incredibly stupid vscode on snap bug
from os import environ
krb5ccname = environ["KRB5CCNAME"]
if "hostfs" in krb5ccname:
    krb5ccname = krb5ccname.replace("/var/lib/snapd/hostfs", "")
    environ["KRB5CCNAME"] = krb5ccname

In [2]:
import ROOT
from analysis_framework import Dataset
from OptimalObservableHelper import OptimalObservableHelper
from AltSetupHandler import AltSetupHandler

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x92fd090


In [3]:
# CLD
# x_angle = 0.030 # rad
# ILD
x_angle = 0.014 # rad
n_threads = 12
# prod = False
prod = True
no_rvec = True
write_outputs = False
# write_outputs = True
# plot_dir_postfix = "-new-cuts"
dataset_path = "data/datasets/reweighted/test.json"
# output_path = "root://eosuser.cern.ch//eos/user/l/lreichen/TGC2/data/reweighted/test"
# output_meta_path = "data/datasets/reweighted"
# output_meta = f"{output_meta_path}/test.json"
# checked_output_meta = f"{output_meta_path}/checked-test.json"
# output_collections = [
#     "true_lep_lvec", "true_nu_lvec", "true_quark1_lvec", "true_quark2_lvec",
#     "iso_lep_lvec", "nu_lvec", "R2Jet_sel1_lvec", "R2Jet_sel2_lvec",
#     ]
# true lvecs do not exist in every df so cannot be explicitly requested...
# urgh but empty snapshots are also not allowed
# output_collections = r"(true_\w+_lvec)|(iso_lep_lvec)|(nu_lvec)|(R2Jet_sel1_lvec)|(R2Jet_sel2_lvec)|(\w*sqme\w*)|(weight\w*)"
# plot_dir = f"plots/pre-selection/test{plot_dir_postfix}"
if prod:
    # dataset_path = "data/datasets/miniDSTs/processed-no-exc-higgs.json"
    # dataset_path = "data/datasets/miniDSTs/processed-no-exc-higgs-min-aa-min-higgs.json"
    # dataset_path = "data/datasets/miniDSTs/min-higgs.json"
    dataset_path = "data/datasets/reweighted/signal-only.json"
    # output_path = "root://eospublic.cern.ch//eos/experiment/clicdp/data/user/l/lreichen/snapshots3/min-higgs-d"
    # output_path = "root://eosuser.cern.ch//eos/user/l/lreichen/TGC2/data/reweighted/signal-only"
    # output_meta_path = "data/datasets/reweighted"
    # output_meta = f"{output_meta_path}/signal-only.json"
    # checked_output_meta = f"{output_meta_path}/checked-signal-only.json"
    # plot_dir = "plots/pre-selection/full"
    # plot_dir = f"plots/pre-selection/min-higgs{plot_dir_postfix}"


In [4]:
ROOT.EnableImplicitMT(n_threads)
# environ["OMP_NUM_THREADS"] = "6"

In [5]:
dataset = Dataset.from_json(dataset_path)

In [6]:
analysis = OptimalObservableHelper(dataset)

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x83480b0


In [7]:
analysis.init_categories()
# check if we missed any processes
print(analysis.is_complete_categorisation())
signal_category = ["4f_sw_sl_signal"]

True


In [8]:
oo_configs = [
    "g1z_pos_1em08",
    "ka_pos_1em08",
    "la_pos_1em08",
    ]
analysis.define_optimal_observables_truth(oo_configs, truth_categories=signal_category)
analysis.define_optimal_observables_reco(oo_configs)
oo_names = [f"O_{c}" for c in oo_configs]

In [9]:
analysis.add_filter("&&".join([f"std::isfinite({oo})" for oo in oo_names]), "finite OO")
analysis.book_reports()

In [10]:
analysis.book_histogram_1D("mc_O_g1z_pos_1em08", "mc_O_g1z_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
analysis.book_histogram_1D("mc_O_ka_pos_1em08", "mc_O_ka_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
analysis.book_histogram_1D("mc_O_la_pos_1em08", "mc_O_la_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)

analysis.book_histogram_1D("O_g1z_pos_1em08", "O_g1z_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
analysis.book_histogram_1D("O_ka_pos_1em08", "O_ka_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
analysis.book_histogram_1D("O_la_pos_1em08", "O_la_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)

In [11]:
alt_setup_handler = AltSetupHandler("""
{
  "SM": {
    "g1z": 1.0,
    "ka": 1.0,
    "la": 0.0
  },
"variations": [
    0.001,
    -0.001,
    1e-04,
    -1e-04,
    1e-08
  ]
}
""", mirror=False, combinations=False)
alt_config_names = list(alt_setup_handler.get_alt_setup().keys())

In [12]:
# book weighted OO sums
for name in ["nominal"] + alt_config_names:
    w_name = f"weight_{name}"
    analysis.book_sum(w_name, w_name, categories=signal_category)
    for oo in oo_names:
        oo_w_name = f"{oo}_{w_name}"
        analysis.define_only_on(signal_category, oo_w_name, f"{oo} * {w_name}")
        analysis.book_sum(oo_w_name, oo_w_name, categories=signal_category)
        analysis.book_histogram_1D(oo_w_name, oo_w_name, ("", "", 250, -2.5, 2.5), categories=signal_category)

        mc_oo_w_name = f"mc_{oo}_{w_name}"
        analysis.define_only_on(signal_category, mc_oo_w_name, f"mc_{oo} * {w_name}")
        analysis.book_sum(mc_oo_w_name, mc_oo_w_name, categories=signal_category)
        analysis.book_histogram_1D(mc_oo_w_name, mc_oo_w_name, ("", "", 250, -2.5, 2.5), categories=signal_category)
        # print(mc_oo_w_name)

In [13]:
analysis.book_oo_matrix([f"mc_{oo}" for oo in oo_names], categories=signal_category)
analysis.book_oo_matrix([f"{oo}" for oo in oo_names], categories=signal_category)

In [14]:
if write_outputs:
    analysis.book_snapshots("events", output_path, output_meta, output_collections, no_rvec=no_rvec)

In [15]:
%%time
analysis.run()

CPU times: user 14.2 s, sys: 829 ms, total: 15 s
Wall time: 3.94 s


In [16]:
analysis.print_reports()
# if write_outputs:
    # analysis.check_snapshots("events", output_path, checked_output_meta)

         4f_sw_sl_signal               4f_sl_bkg
        10244400 (1e+04)           33239 (6e+02) All
        10236856 (1e+04)           33227 (6e+02) finite OO
                    1.00                    1.00 efficiency



In [17]:
analysis.draw_histogram("mc_O_g1z_pos_1em08", categories=signal_category)
analysis.draw_histogram("O_g1z_pos_1em08", categories=signal_category)

analysis.draw_histogram("mc_O_ka_pos_1em08", categories=signal_category)
analysis.draw_histogram("O_ka_pos_1em08", categories=signal_category)

analysis.draw_histogram("mc_O_la_pos_1em08", categories=signal_category)
analysis.draw_histogram("O_la_pos_1em08", categories=signal_category)

(<cppyy.gbl.THStack object at 0x13cdaf60>,
 <cppyy.gbl.TCanvas object at 0x1251e500>)

In [18]:
analysis.compare_histograms_unscaled(["mc_O_g1z_pos_1em08", "O_g1z_pos_1em08"], categories=signal_category)
analysis.compare_histograms_unscaled(["mc_O_ka_pos_1em08", "O_ka_pos_1em08"], categories=signal_category)
analysis.compare_histograms_unscaled(["mc_O_la_pos_1em08", "O_la_pos_1em08"], categories=signal_category)

In [19]:
analysis.compare_histograms_unscaled(["mc_O_g1z_pos_1em08_weight_nominal"] + [f"mc_O_g1z_pos_1em08_weight_{c}" for c in alt_config_names if "g1z" in c], categories=signal_category)

In [20]:
# calculate means etc.
w_sums = {}
oo_sums = {}
mc_oo_sums = {}
oo_means = {}
mc_oo_means = {}
for name in ["nominal"] + alt_config_names:
    w_name = f"weight_{name}"
    w_sum = analysis.get_sum(w_name, categories=signal_category)
    w_sums[w_name] = w_sum
    for oo in oo_names:
        oo_w_name = f"{oo}_{w_name}"
        oo_sums[oo_w_name] = analysis.get_sum(oo_w_name, categories=signal_category)
        oo_means[oo_w_name] = oo_sums[oo_w_name] / w_sum
        mc_oo_w_name = f"mc_{oo}_{w_name}"
        mc_oo_sums[mc_oo_w_name] = analysis.get_sum(mc_oo_w_name, categories=signal_category)
        mc_oo_means[mc_oo_w_name] = mc_oo_sums[mc_oo_w_name] / w_sum

In [21]:
print(alt_config_names)

['g1z_pos_1em03', 'ka_pos_1em03', 'la_pos_1em03', 'g1z_neg_1em03', 'ka_neg_1em03', 'la_neg_1em03', 'g1z_pos_1em04', 'ka_pos_1em04', 'la_pos_1em04', 'g1z_neg_1em04', 'ka_neg_1em04', 'la_neg_1em04', 'g1z_pos_1em08', 'ka_pos_1em08', 'la_pos_1em08']


In [22]:
graphs = {}

for oo in oo_names:
    for par in ["g1z", "ka", "la"]:
        vars = [var for var in alt_config_names if par in var]
        nominal = mc_oo_means[f"mc_{oo}_weight_nominal"]
        x = [AltSetupHandler.get_var_from_name_1d(var) for var in vars]
        y = [mc_oo_means[f"mc_{oo}_weight_{var}"] / nominal for var in vars]
        x.append(0.)
        y.append(1.)
        vals = sorted(zip(x, y), key=lambda a: a[0])
        print(vals)
        g = ROOT.TGraph()
        for v in vals:
            g.AddPoint(v[0], v[1])
        g.SetTitle(f"{oo};{par}; #overline{{O}} / #overline{{O}}_{{0}}")
        graphs[f"mc_{oo}_{par}"] = g

[(-0.001, 1.014236572521722), (-0.0001, 1.0014242244405476), (0.0, 1.0), (1e-08, 0.99999985757127), (0.0001, 0.99857564983558), (0.001, 0.9857508551072715)]
[(-0.001, 0.9997534511006655), (-0.0001, 0.9999758670878678), (0.0, 1.0), (1e-08, 1.000000002407491), (0.0001, 1.0000240169064796), (0.001, 1.0002349483379391)]
[(-0.001, 0.9919059867782597), (-0.0001, 0.9991902969520958), (0.0, 1.0), (1e-08, 1.0000000809736462), (0.0001, 1.0008097698515532), (0.001, 1.0081006935797991)]
[(-0.001, 0.9996642592848691), (-0.0001, 0.9999641296479308), (0.0, 1.0), (1e-08, 1.000000003612554), (0.0001, 1.0000363806872141), (0.001, 1.0003867741765062)]
[(-0.001, 1.006378638676182), (-0.0001, 1.0006369967841149), (0.0, 1.0), (1e-08, 0.9999999363099605), (0.0001, 0.999363195960648), (0.001, 0.99364063579423)]
[(-0.001, 0.999013237322083), (-0.0001, 0.9998995919609406), (0.0, 1.0), (1e-08, 1.0000000100600444), (0.0001, 1.0001007928095917), (0.001, 1.001025239680437)]
[(-0.001, 1.0093238867516356), (-0.0001, 

In [23]:
canvases = {}
for k, g in graphs.items():
    c = ROOT.TCanvas()
    g.Draw("alp")
    c.Draw()
    canvases[k] = c

In [24]:
mc_M = analysis.get_oo_matrix([f"mc_{oo}" for oo in oo_names], categories=signal_category)

In [25]:
import numpy as np
from itertools import combinations_with_replacement
mc_C_tilde = np.zeros((3,3))
for k, (i, j) in enumerate(combinations_with_replacement(range(3), 2)):
    mc_C_tilde[i, j] = mc_M[k]
    mc_C_tilde[j, i] = mc_M[k]
print(mc_C_tilde)

[[15284586.19559216  -182595.51780891 -8722922.82095906]
 [ -182595.51780891  4571090.5833652   -778454.48403762]
 [-8722922.82095906  -778454.48403762 24734389.7752713 ]]


In [26]:
print(np.sqrt(np.diag(np.linalg.inv(mc_C_tilde))))

[0.00028674 0.00046987 0.00022596]


In [27]:
mc_oo_means_vec = np.asarray([mc_oo_means[f"mc_{oo}_weight_nominal"] for oo in oo_names])
mc_C = mc_C_tilde / w_sums["weight_nominal"] - np.outer(mc_oo_means_vec, mc_oo_means_vec)
print(mc_C)

[[ 1.48226319 -0.02505489 -0.84269694]
 [-0.02505489  0.44172256 -0.06977149]
 [-0.84269694 -0.06977149  2.40802942]]


In [28]:
print(np.sqrt(np.diag(np.linalg.inv(mc_C)) / w_sums["weight_nominal"]))

[0.00028751 0.00047245 0.00022598]


In [29]:
M = analysis.get_oo_matrix([f"{oo}" for oo in oo_names], categories=signal_category)
C_tilde = np.zeros((3,3))
for k, (i, j) in enumerate(combinations_with_replacement(range(3), 2)):
    C_tilde[i, j] = M[k]
    C_tilde[j, i] = M[k]
print(C_tilde)
print(np.sqrt(np.diag(np.linalg.inv(C_tilde))))
oo_means_vec = np.asarray([oo_means[f"{oo}_weight_nominal"] for oo in oo_names])
C = C_tilde / w_sums["weight_nominal"] - np.outer(oo_means_vec, oo_means_vec)
print(C)
print(np.sqrt(np.diag(np.linalg.inv(C)) / w_sums["weight_nominal"]))

[[ 7351701.02771668  1583513.11599528 -3750477.9751206 ]
 [ 1583513.11599528  4670479.75327239   -37805.86981575]
 [-3750477.9751206    -37805.86981575 15288329.33052928]]
[0.00041166 0.00048308 0.00027484]
[[ 0.71807801  0.15441738 -0.36708273]
 [ 0.15441738  0.45535297 -0.00603798]
 [-0.36708273 -0.00603798  1.48727225]]
[0.00041178 0.00048339 0.0002755 ]
